이 노트북은 [케라스 창시자에게 배우는 딥러닝 2판](https://tensorflow.blog/kerasdl2/)의 예제 코드를 담고 있습니다.

<table align="left">
    <tr>
        <td>
            <a href="https://colab.research.google.com/github/rickiepark/deep-learning-with-python-2nd/blob/main/chapter12_part01_text-generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
        </td>
    </tr>
</table>

# 생성 모델을 위한 딥러닝

## 텍스트 생성

### 시퀀스 생성을 위한 딥러닝 모델의 간단한 역사

### 시퀀스 데이터를 어떻게 생성할까?

### 샘플링 전략의 중요성

**다른 온도 값을 사용하여 확률 분포의 가중치 바꾸기**

In [1]:
import numpy as np
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

### 케라스를 사용한 텍스트 생성 모델 구현

#### 데이터 준비

**IMDB 영화 리뷰 데이터셋 다운로드하고 압축 풀기**

In [2]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2024-05-28 01:52:07--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  6.02MB/s    in 23s     

2024-05-28 01:52:31 (3.44 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



**텍스트 파일(한 파일 = 한 샘플)에서 데이터셋 만들기**

In [3]:
import tensorflow as tf
from tensorflow import keras
dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb", label_mode=None, batch_size=256)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))

Found 100006 files belonging to 1 classes.


**`TextVectorization` 층 준비하기**

In [4]:
from tensorflow.keras.layers import TextVectorization

sequence_length = 100
vocab_size = 15000
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(dataset)

**언어 모델링 데이터셋 만들기**

In [5]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    x = vectorized_sequences[:, :-1]
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

In [6]:
# prompt: IM_dataset에서 첫번째 샘플 출력해줘

for x, y in lm_dataset.take(1):
  print(x[0])
  print(y[0])


tf.Tensor(
[  252 12486     4   969     5     1  4138    10   212  5489     9     7
    22    14   155    14    10  1836   195     9     7   123  1348  1505
  1851   124   530    45  1111 13999   110     1  4061    30  2193 12574
     3  8011  5215   735   117     2  3056  2165    36  1166     1     7
     8  1218 13213    29    31     1    11     3     1     1     1  1865
     1  4138  2897    22    12   224    77 11235  6717     1  3472  1424
   104     9   172     2  4075   346     5   145   876    15     1  1736
    13   521     3     2  1403   140    26  2370     5   717   170     4
   935   215    20], shape=(99,), dtype=int64)
tf.Tensor(
[12486     4   969     5     1  4138    10   212  5489     9     7    22
    14   155    14    10  1836   195     9     7   123  1348  1505  1851
   124   530    45  1111 13999   110     1  4061    30  2193 12574     3
  8011  5215   735   117     2  3056  2165    36  1166     1     7     8
  1218 13213    29    31     1    11     3     1     1 

#### 트랜스포머 기반의 시퀀스-투-시퀀스 모델

In [7]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    # def get_causal_attention_mask(self, inputs):
    #     input_shape = tf.shape(inputs)
    #     batch_size, sequence_length = input_shape[0], input_shape[1]
    #     i = tf.range(sequence_length)[:, tf.newaxis]
    #     j = tf.range(sequence_length)
    #     mask = tf.cast(i >= j, dtype="int32")
    #     mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
    #     mult = tf.concat(
    #         [tf.expand_dims(batch_size, -1),
    #          tf.constant([1, 1], dtype=tf.int32)], axis=0)
    #     return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        # causal_mask = self.get_causal_attention_mask(inputs)
        # if mask is not None:
        #     padding_mask = tf.cast(
        #         mask[:, tf.newaxis, :], dtype="int32")
        #     padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            use_causal_mask=True)
            # attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs
            # attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

**간단한 트랜스포머 기반 언어 모델**

In [8]:
from tensorflow.keras import layers
embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

### 가변 온도 샘플링을 사용한 텍스트 생성 콜백

**텍스트 생성 콜백**

In [9]:
import numpy as np

tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,
                 generate_length,
                 model_input_length,
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            sentence = self.prompt
            for i in range(self.generate_length):
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                next_token = sample_next(predictions[0, i, :], temperature)
                sampled_token = tokens_index[next_token]
                sentence += " " + sampled_token
            print(sentence)

prompt = "This movie"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

**언어 모델 훈련하기**

In [10]:
# 코랩에서 정상 실행만 확인하기 위해 에포크 횟수를 200에서 10으로 줄입니다
model.fit(lm_dataset, epochs=10,  # 200
          callbacks=[text_gen_callback])

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 5.9231== Generating with temperature 0.2
This movie movie was was a a lot lot of of the the movie first was time a i few was years a ago lot and of i the was [UNK] the and film i was was the a movie lot was of the the first best part part of of the
== Generating with temperature 0.5
This movie movie was was a lucky good so movie i was was not popular a i [UNK] was [UNK] so and horrible i acting was and a i man was who a was lot made of for the the first way time that i i liked have the seen first it
== Generating with temperature 0.7
This movie film was is awful one movies of like the about storyline it but was it a wasnt waste in of much the better [UNK] than convoluted a [UNK] few movies minutes presented of the the movie spells is of a [UNK] charming and and [UNK] the in role the in
== Generating with temperature 1.0
This movie movie happened and being [UNK] michael the alot smaller a oscar collection and acclaime

### 정리